## 자연어 처리

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Oshima
- 부동산 사건 사고가 표기된 info 컬럼 데이터의 주요 키워드 추출  

In [3]:
import pandas as pd

In [4]:
colab_path = '/content/drive/MyDrive/Google Colab/Data_Project/Airbnb_Project/Dataset_Processing/yunyoung/data'
vscode_path = './data'
open = pd.read_csv(colab_path + '/open_data_add_latlng.csv', index_col=0)
airbnb = pd.read_csv(colab_path+'/airbnb_data_add_latlng.csv', index_col=0)
oshima_data = pd.read_csv(colab_path+'/oshima_data_add_latlng.csv', index_col=0)
oshima_data.head()

,district,address,info,date,full name,latitude,longitude
0,新宿区,東京都新宿区新宿三丁目22-7指田ビル地下1階,死体発見,平成24年11月5日,東京都 新宿区 新宿 三丁目 22番 7号,35.693264,139.701309
1,新宿区,東京都新宿区西新宿一丁目3-9,飛び降り自殺,令和4年,東京都 新宿区 西新宿 一丁目 3番 9号,35.693218,139.698166
2,新宿区,東京都新宿区歌舞伎町一丁目27-5 中台ビル,2階サウナロイヤル 死体発見,平成20年1月5日,東京都 新宿区 歌舞伎町 一丁目 27番 5号,35.695450,139.700623
3,新宿区,東京都新宿区歌舞伎町一丁目20-2アパホテル新宿歌舞伎町タワー↓,飛び降り自殺,令和3年5月11日,東京都 新宿区 歌舞伎町 一丁目 20番 2号,35.695652,139.701401
4,新宿区,東京都新宿区西新宿七丁目5-9プロスペクト・アクス・ザ・タワー新宿2008,首吊り自殺,平成27年9月26日,東京都 新宿区 西新宿 七丁目 5番 9号,35.696812,139.698044


In [7]:
oshima_data['info'].value_counts()[-10:]

二階、階段をあがって二番目の部屋\n生活保護者の孤高死                                                                        1
60代男性が孤高死                                                                                          1
建築完成後入居前、飛び降り自殺                                                                                    1
50代男性が病死                                                                                           1
桐ヶ丘中学校男子生徒飛び降り自殺                                                                                   1
1階 角部屋 南向き1K（洋室6帖、K1帖） 専有面積15.5㎡\n告知事項あり\n生活保護相談（60代まで）                                            1
やよい荘１０１号室 死体発見（死後３週間前後と推定）                                                                         1
賃貸一戸建て\n鉄骨造 2階建（屋上あり） 主要採光面：南向き5DK（ DK8帖、洋室9帖、洋室4帖、和室6畳、和室6畳、和室6畳） 専有面積135.12㎡\n告知事項有\n定期借家（1年）    1
死体発見 ．                                                                                             1
多摩ニュータウン永山団地4-2-22号棟105号室\n特別募集住宅                                                          

### 데이터 살펴보기

In [ ]:
import ast
from collections import Counter
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
import plotly.express as px

In [ ]:
pip install ipadic==1.0.0a2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 13.1 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for ipadic: filename=ipadic-1.0.0a2-py3-none-any.whl size=13556740 sha256=d3048b4ebfcc6a83e236b3142eb25f7b56ed09333b8e7d19b3a4d52f3b777f8a
  Stored in directory: /Users/yunyoung/Library/Caches/pip/wheels/44/f9/02/862651969e5626d2fcc219224a366568ca5cce1eb70c2a48b4
Successfully built ipadic
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import matplotlib.font_manager as font_manager

font_list = font_manager.findSystemFonts(fontpaths=None, fontext='ttf')
font_list[:10]

['/System/Library/Fonts/Supplemental/NotoSansMarchen-Regular.ttf',
 '/System/Library/Fonts/Supplemental/Galvji.ttc',
 '/Library/Fonts/Arial Unicode.ttf',
 '/System/Library/Fonts/Supplemental/SignPainter.ttc',
 '/System/Library/Fonts/Supplemental/DevanagariMT.ttc',
 '/System/Library/Fonts/Supplemental/NotoSansImperialAramaic-Regular.ttf',
 '/System/Library/Fonts/Supplemental/NotoSansBhaiksuki-Regular.ttf',
 '/Users/yunyoung/Library/Fonts/NanumBrush.ttf',
 '/System/Library/Fonts/Supplemental/NotoSansSiddham-Regular.ttf',
 '/System/Library/Fonts/Supplemental/Times New Roman Bold.ttf']

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from collections import Counter
import MeCab

m = MeCab.Tagger()


def extract_nouns(text):
    node = m.parseToNode(text)
    nouns = []
    while node:
        if node.feature.split(',')[0] == '名詞':  # Extract only nouns
            nouns.append(node.surface)
        node = node.next
    return nouns

# Apply the function to each row in the 'info' column
oshima_data['nouns'] = oshima_data['info'].apply(extract_nouns)
oshima_data[:10]

,district,address,info,date,full name,latitude,longitude,nouns
0,新宿区,東京都新宿区新宿三丁目22-7指田ビル地下1階,死体発見,平成24年11月5日,東京都 新宿区 新宿 三丁目 22番 7号,35.693264,139.701309,"[死体, 発見]"
1,新宿区,東京都新宿区西新宿一丁目3-9,飛び降り自殺,令和4年,東京都 新宿区 西新宿 一丁目 3番 9号,35.693218,139.698166,[自殺]
2,新宿区,東京都新宿区歌舞伎町一丁目27-5 中台ビル,2階サウナロイヤル 死体発見,平成20年1月5日,東京都 新宿区 歌舞伎町 一丁目 27番 5号,35.695450,139.700623,"[2, 階, サウナ, ロイヤル, 死体, 発見]"
3,新宿区,東京都新宿区歌舞伎町一丁目20-2アパホテル新宿歌舞伎町タワー↓,飛び降り自殺,令和3年5月11日,東京都 新宿区 歌舞伎町 一丁目 20番 2号,35.695652,139.701401,[自殺]
4,新宿区,東京都新宿区西新宿七丁目5-9プロスペクト・アクス・ザ・タワー新宿2008,首吊り自殺,平成27年9月26日,東京都 新宿区 西新宿 七丁目 5番 9号,35.696812,139.698044,"[首吊り, 自殺]"
5,新宿区,東京都新宿区西新宿七丁目5-25,20階\n心理的瑕疵あり,不明,東京都 新宿区 西新宿 七丁目 5番 25号,35.697594,139.698837,"[20, 階, 心理, 瑕疵, あり]"
6,新宿区,東京都新宿区歌舞伎町二丁目35-6,コインロッカー内に乳児の腐乱死体。,平成30年6月12日,東京都 新宿区 歌舞伎町 二丁目 35番 6号,35.696503,139.702469,"[コイン, ロッカー, 乳児, 腐乱, 死体]"
7,新宿区,東京都新宿区歌舞伎町二丁目8-2,902号室で自殺あり。,2020年7月初旬,東京都 新宿区 歌舞伎町 二丁目 8番 2号,35.694901,139.705887,"[902, 号, 自殺, あり]"
8,新宿区,東京都新宿区新宿五丁目16-10,伊勢丹テナント関係者の首吊り自殺,2008年頃,東京都 新宿区 新宿 五丁目 16番 10号,35.693150,139.706360,"[伊勢丹, テナント, 関係, 首吊り, 自殺]"
9,新宿区,東京都新宿区新宿五丁目18-20 ビジネスホテルミラベル,309号室 首吊り自殺,平成20年12月4日,東京都 新宿区 新宿 五丁目 18番 20号,35.693932,139.706482,"[309, 号, 首吊り, 自殺]"


### 일본어 -> 한국어 번역

In [5]:
!pip install deep-translator

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 1.4 MB/s eta 0:00:00


In [6]:
from deep_translator import GoogleTranslator

translator = GoogleTranslator(source='ja', target='ko')

def translate(x):
    translated_text = translator.translate(x)
    return translated_text

In [7]:
from tqdm.notebook import tqdm_notebook

tqdm_notebook()

oshima_data['translate'] = ""

for i, row in tqdm_notebook(oshima_data.iterrows(), total=len(oshima_data)):
    oshima_data['translate'][i] = translate(row['info'])

0it [00:00, ?it/s]

  0%|          | 0/4833 [00:00<?, ?it/s]

<ipython-input-7-12f0b8a1dd51>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  oshima_data['translate'][i] = translate(row['info'])


In [3]:
oshima_data.to_csv('translate.csv')


### 태깅하기

In [1]:
import pandas as pd 

oshima_kr = pd.read_csv('../yunyoung/data/translate.csv', index_col=0)
oshima_kr

,district,address,info,date,full name,latitude,longitude,translate
0,新宿区,東京都新宿区新宿三丁目22-7指田ビル地下1階,死体発見,平成24年11月5日,東京都 新宿区 新宿 三丁目 22番 7号,35.693264,139.701309,시체 발견
1,新宿区,東京都新宿区西新宿一丁目3-9,飛び降り自殺,令和4年,東京都 新宿区 西新宿 一丁目 3番 9号,35.693218,139.698166,뛰어 내리는 자살
2,新宿区,東京都新宿区歌舞伎町一丁目27-5 中台ビル,2階サウナロイヤル 死体発見,平成20年1月5日,東京都 新宿区 歌舞伎町 一丁目 27番 5号,35.695450,139.700623,2층 사우나 로얄 시체 발견
3,新宿区,東京都新宿区歌舞伎町一丁目20-2アパホテル新宿歌舞伎町タワー↓,飛び降り自殺,令和3年5月11日,東京都 新宿区 歌舞伎町 一丁目 20番 2号,35.695652,139.701401,뛰어 내리는 자살
4,新宿区,東京都新宿区西新宿七丁目5-9プロスペクト・アクス・ザ・タワー新宿2008,首吊り自殺,平成27年9月26日,東京都 新宿区 西新宿 七丁目 5番 9号,35.696812,139.698044,목 매달아 자살
...,...,...,...,...,...,...,...,...
4844,多摩市,東京都八王子市大塚640-13ハウスオブパレ,隣室で謎の死。当時私が住んでいた307号室に謎の黒い影が頻繁に出て来るようになりました。,2006年4月頃,東京都 八王子市 大塚 640番地,35.637493,139.422913,이웃실에서 수수께끼의 죽음. 당시 제가 살았던 307호실에 수수께끼의 검은 그림자가...
4845,多摩市,東京都八王子市大塚769-8,代表取締役が恋人のゲイと喧嘩し、刺殺事故,2015年頃,東京都 八王子市 大塚 769番地,35.638660,139.424469,대표 이사가 연인의 게이와 싸우고 자살 사고
4846,多摩市,東京都多摩市和田2009-1,飛び降り自殺,2018年2月,東京都 多摩市 和田 2009番地,35.639858,139.427063,뛰어 내리는 자살
4847,多摩市,東京都八王子市大塚,1993年ごろかな。10号館あたりで学生が飛び降り自殺をした。何階からかは忘れたけど、渡り廊...,平成24年10月1日,東京都 八王子市 大塚,35.639523,139.418701,1993년경일까. 10호관 근처에서 학생이 뛰어내려 자살을 했다. 몇 층부터는 잊었...


#### 주소 기준 중복값 제거

In [2]:
oshima_kr_unique = oshima_kr.drop_duplicates(subset=['address'])

In [3]:
oshima_kr_unique.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3644 entries, 0 to 4836
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   district   3644 non-null   object 
 1   address    3644 non-null   object 
 2   info       3644 non-null   object 
 3   date       3644 non-null   object 
 4   full name  3644 non-null   object 
 5   latitude   3644 non-null   float64
 6   longitude  3644 non-null   float64
 7   translate  3644 non-null   object 
dtypes: float64(2), object(6)
memory usage: 256.2+ KB


In [10]:
df_unique = oshima_kr.drop_duplicates(subset=['address'])
df_unique.reset_index(drop=True, inplace = True)
df_unique

,district,address,info,date,full name,latitude,longitude,translate
0,新宿区,東京都新宿区新宿三丁目22-7指田ビル地下1階,死体発見,平成24年11月5日,東京都 新宿区 新宿 三丁目 22番 7号,35.693264,139.701309,시체 발견
1,新宿区,東京都新宿区西新宿一丁目3-9,飛び降り自殺,令和4年,東京都 新宿区 西新宿 一丁目 3番 9号,35.693218,139.698166,뛰어 내리는 자살
2,新宿区,東京都新宿区歌舞伎町一丁目27-5 中台ビル,2階サウナロイヤル 死体発見,平成20年1月5日,東京都 新宿区 歌舞伎町 一丁目 27番 5号,35.695450,139.700623,2층 사우나 로얄 시체 발견
3,新宿区,東京都新宿区歌舞伎町一丁目20-2アパホテル新宿歌舞伎町タワー↓,飛び降り自殺,令和3年5月11日,東京都 新宿区 歌舞伎町 一丁目 20番 2号,35.695652,139.701401,뛰어 내리는 자살
4,新宿区,東京都新宿区西新宿七丁目5-9プロスペクト・アクス・ザ・タワー新宿2008,首吊り自殺,平成27年9月26日,東京都 新宿区 西新宿 七丁目 5番 9号,35.696812,139.698044,목 매달아 자살
...,...,...,...,...,...,...,...,...
3639,杉並区,東京都杉並区善福寺三丁目6-13,心理的瑕疵あり,不明,東京都 杉並区 善福寺 三丁目 6番 13号,35.716324,139.592682,심리적 하자 있음
3640,杉並区,東京都杉並区上井草三丁目37-15プチコ－トＵ201号室,「心理的瑕疵あり」と説明あり (LIFULL HOME'S - エイブル上石神井店),不明,東京都 杉並区 上井草 三丁目 37番 15号,35.723637,139.600052,"""심리적 하자 있음""이라고 설명 있음 (LIFULL HOME'S - 에이블 카미이시..."
3641,多摩市,東京都多摩市諏訪四丁目3-3都営諏訪団地4-3-3号棟105,自殺&死体発見,平成24年8月7日,東京都 多摩市 諏訪 四丁目 3番,35.622181,139.458511,자살 & 시체 발견
3642,多摩市,東京都多摩市永山四丁目3-14,多摩ニュータウン永山団地4-3-14号棟102号室\n特別募集住宅,不明,東京都 多摩市 永山 四丁目 3番,35.617443,139.450256,타마 뉴타운 영산단지 4-3-14호동 102호실\n특별 모집 주택


In [11]:
df_unique['info'].unique()

array(['死体発見', '飛び降り自殺', '2階サウナロイヤル 死体発見', ..., '自殺&死体発見',
       '多摩ニュータウン永山団地4-3-14号棟102号室\n特別募集住宅',
       '多摩ニュータウン永山団地4-2-22号棟105号室\n特別募集住宅'], dtype=object)

In [ ]:
# 전처리 함수
def preprocess_sentence(sentence, remove_stopwords = True):
    sentence = BeautifulSoup(sentence, "lxml").text # <br />, <a href = ...> 등의 html 태그 제거
    sentence = re.sub(r'\([^)]*\)', '', sentence) # 괄호로 닫힌 문자열  제거 Ex) my husband (and myself) for => my husband for
    sentence = re.sub('"','', sentence) # 쌍따옴표 " 제거
    sentence = ' '.join([contractions[t] if t in contractions else t for t in sentence.split(" ")]) # 약어 정규화
    sentence = re.sub("[^a-zA-Z]", " ", sentence) # 영어 외 문자(숫자, 특수문자 등) 공백으로 변환
\
    # 불용어 제거 (Text)
    if remove_stopwords:
        tokens = ' '.join(word for word in sentence.split() if not word in stop_words if len(word) > 1)
    # 불용어 미제거 (Summary)
    else:
        tokens = ' '.join(word for word in sentence.split() if len(word) > 1)
    return tokens
